In [1]:
import ipsuite as ips

2023-10-24 19:54:43,228 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


DFT based on https://www.sciencedirect.com/science/article/pii/S1386142521004455#b0030

In [2]:
cp2k_shell = "cp2k_shell.ssmp"
# cp2k_shell = "mpirun -np 12 /home/linux38_i1/schaefer/miniconda3/envs/dmso/bin/cp2k_shell.psmp"

# Auxiliary Nodes

In [3]:
thermostat = ips.calculators.LangevinThermostat(
    temperature=298.15, friction=0.01, time_step=0.5
)

uncertainty_check = ips.analysis.ThresholdCheck(
    value="energy_uncertainty", max_value=2.0, larger_only=True
)

mapping = ips.geometry.BarycenterMapping(data=None)
temperature_oszillator = ips.calculators.TemperatureOscillatingRampModifier(
    end_temperature=450,  # boiling around 460
    start_temperature=270,  # melting around 290
    num_oscillations=10,
    temperature_amplitude=150,
)

box_oszillator = ips.calculators.BoxOscillatingRampModifier(
    cell_amplitude=1,
    num_oscillations=3,
)

eq_box_oszillator = ips.calculators.BoxOscillatingRampModifier(
    end_cell=16.511,
    cell_amplitude=1,
    num_oscillations=3,
)

# Initial Training Data

In [4]:
with ips.Project(automatic_node_names=True) as project:
    mol = ips.configuration_generation.SmilesToAtoms(smiles="CS(=O)C")

    # Create a box of atoms.
    packmol = ips.configuration_generation.Packmol(
        data=[mol.atoms], count=[38], density=1095.2
    )

    # Define the CP2K calculations
    cp2k = ips.calculators.CP2KSinglePoint(
        data=packmol.atoms,
        cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
        cp2k_shell=cp2k_shell,
    )

    geopt = ips.calculators.ASEGeoOpt(
        model=cp2k,
        data=packmol.atoms,
        optimizer="BFGS",
        run_kwargs={"fmax": 0.5},
    )

    test_selection = ips.configuration_selection.RandomSelection(
        data=geopt.atoms, n_configurations=20
    )

    train_data = test_selection.excluded_atoms
    test_data = test_selection.atoms

# First AL cycles

In [5]:
models = []

In [6]:
with project:
    for cycle in range(6):
        with project.group(f"AL_{cycle}") as group:

            # Define the ML model
            model1 = ips.models.Apax(
                data=train_data,
                validation_data=test_data,
                config="config/apax_1.yaml" if cycle < 5 else "config/apax_3.yaml",
            )
            model2 = ips.models.Apax(
                data=train_data,
                validation_data=test_data,
                config="config/apax_2.yaml" if cycle < 5 else "config/apax_4.yaml",
            )

            ensemble_model = ips.models.EnsembleModel(models=[model1, model2])

            models.append(ensemble_model)

            md = ips.calculators.ASEMD(
                data=geopt.atoms,
                data_id=-1,
                model=ensemble_model,
                thermostat=thermostat,
                checker_list=[uncertainty_check],
                steps=50000,
                sampling_rate=1,
            )

            train_data_selection = ips.configuration_selection.ThresholdSelection(
                data=md, n_configurations=10, min_distance=10
            )

            test_data_selection = ips.configuration_selection.RandomSelection(
                data=md,
                n_configurations=5,
                exclude_configurations=train_data_selection.selected_configurations,
            )

            train_data += ips.calculators.CP2KSinglePoint(
                data=train_data_selection,
                cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                cp2k_shell=cp2k_shell,
                wfn_restart_node=cp2k,
            ).atoms

            test_data += ips.calculators.CP2KSinglePoint(
                data=test_data_selection,
                cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                cp2k_shell=cp2k_shell,
                wfn_restart_node=cp2k,
            ).atoms

            md_forces_uncertainty = ips.analysis.ForcesUncertaintyHistogram(
                data=md.atoms
            )
            md_energy_uncertainty = ips.analysis.EnergyUncertaintyHistogram(
                data=md.atoms
            )

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
2023-10-24 19:54:45.559912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-24 19:54:45.559936: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-24 19:54:45.559951: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-24 19:54:46.492708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Bootstrap Data

In [7]:
with project:
    with project.group("bootstrap_0") as group:
        bootstrap_train_data = (
            ips.bootstrap.RotateMolecules(
                data=geopt.atoms,
                data_id=-1,
                n_configurations=10,
                maximum=10 * 3.1415 / 180,  # deg max rotation
                include_original=False,
                seed=1,
            ).atoms
            + ips.bootstrap.TranslateMolecules(
                data=geopt.atoms,
                data_id=-1,
                n_configurations=10,
                maximum=0.2,  # Ang max molecular displacement
                include_original=False,
                seed=1,
            ).atoms
        )

        bootstrap_test_data = (
            ips.bootstrap.RotateMolecules(
                data=geopt.atoms,
                data_id=-1,
                n_configurations=5,
                maximum=10 * 3.1415 / 180,  # deg max rotation
                include_original=False,
                seed=2,
                name="RotateMolecules_test",
            ).atoms
            + ips.bootstrap.TranslateMolecules(
                data=geopt.atoms,
                data_id=-1,
                n_configurations=5,
                maximum=0.2,  # Ang max molecular displacement
                include_original=False,
                seed=2,
                name="TranslateMolecules_test",
            ).atoms
        )

        train_data += ips.calculators.CP2KSinglePoint(
            data=bootstrap_train_data,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell=cp2k_shell,
            wfn_restart_node=cp2k,
        ).atoms

        test_data += ips.calculators.CP2KSinglePoint(
            data=bootstrap_test_data,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell=cp2k_shell,
            wfn_restart_node=cp2k,
        ).atoms

        model1 = ips.models.Apax(
            data=train_data,
            validation_data=test_data,
            config="config/apax_3.yaml",
        )
        model2 = ips.models.Apax(
            data=train_data,
            validation_data=test_data,
            config="config/apax_4.yaml",
        )

        ensemble_model = ips.models.EnsembleModel(models=[model1, model2])

        models.append(ensemble_model)

# Volume Scans

In [8]:
with project:
    with project.group("volume_scan") as volume_scans:
        vs_md = ips.calculators.ASEMD(
            data=geopt.atoms,
            data_id=-1,
            model=ensemble_model,
            thermostat=thermostat,
            checker_list=[uncertainty_check],
            steps=250,
            sampling_rate=10,
        )

        volume_scan = ips.analysis.BoxScale(
            data=vs_md.atoms,
            mapping=mapping,
            model=ensemble_model,
            start=0.9,
            stop=1.7,
            num=50,
            data_id=-1,
        )

        vs_cp2k = ips.calculators.CP2KSinglePoint(
            data=volume_scan.atoms,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell="cp2k_shell.ssmp",
            wfn_restart_node=cp2k,
        )

# Biased MD

In [9]:
with project:
    with project.group("bootstrap_1") as group:
        md = ips.calculators.ASEMD(
            data=geopt.atoms,
            data_id=-1,
            model=ensemble_model,
            thermostat=thermostat,
            checker_list=[uncertainty_check],
            modifier=[temperature_oszillator, box_oszillator],
            steps=10000,
            sampling_rate=100,
        )

        train_data_selection_1 = ips.configuration_selection.ThresholdSelection(
            data=md, n_configurations=20, min_distance=10
        )
        train_data_selection_2 = ips.configuration_selection.RandomSelection(
            data=train_data_selection_1.excluded_atoms, n_configurations=80
        )
        test_data_selection = ips.configuration_selection.RandomSelection(
            data=train_data_selection_2.excluded_atoms, n_configurations=20
        )

        train_data += ips.calculators.CP2KSinglePoint(
            data=train_data_selection_1.atoms + train_data_selection_2.atoms,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell=cp2k_shell,
            wfn_restart_node=cp2k,
        ).atoms

        test_data += ips.calculators.CP2KSinglePoint(
            data=test_data_selection.atoms,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell=cp2k_shell,
            wfn_restart_node=cp2k,
        ).atoms

# More AL

In [10]:
def remove_fused_molecules(train_data, test_data):
    train_data = ips.configuration_selection.IndexSelection(data= train_data, indices=slice(10, -1)).atoms
    test_data = ips.configuration_selection.IndexSelection(data= test_data, indices=slice(4, -1)).atoms
    return train_data, test_data

In [11]:
with project:
    for cycle in range(6):
        with project.group(f"AL2_{cycle}") as al2_group:
            if cycle == 2:
                # remove fused molecules
                train_data, test_data = remove_fused_molecules(train_data, test_data)

            model1 = ips.models.Apax(
                data=train_data,
                validation_data=test_data,
                config="config/apax_al2_1.yaml",
            )
            model2 = ips.models.Apax(
                data=train_data,
                validation_data=test_data,
                config="config/apax_al2_2.yaml",
            )

            ensemble_model = ips.models.ApaxEnsemble(models=[model1, model2])

            models.append(ensemble_model)

            # get starting structure
            model_geopt = ips.calculators.ASEGeoOpt(
                model=ensemble_model,
                data=md.atoms,
                data_id=-5 if cycle > 1 else -1,
                optimizer="BFGS",
                run_kwargs={"fmax": 1.0 if cycle > 1 else 0.5},
                checker_list=[uncertainty_check] if cycle > 1 else None,
            )

            ref_geopt = ips.calculators.ASEGeoOpt(
                model=cp2k,
                data=model_geopt.atoms,
                data_id=-1,
                optimizer="BFGS",
                run_kwargs={"fmax": 2.0 if cycle > 1 else 1.0},
            )

            md = ips.calculators.ASEMD(
                data=ref_geopt.atoms if cycle > 1 else md.atoms,
                data_id=-1,
                model=ensemble_model,
                thermostat=thermostat,
                checker_list=[uncertainty_check],
                modifier=[temperature_oszillator, eq_box_oszillator if cycle > 1 else box_oszillator],
                steps=1_000_000,
                sampling_rate=40 if cycle > 1 else 100,
            )

            if cycle > 1:
                # throw out last atoms from ASEMD
                md = ips.configuration_selection.IndexSelection(data=md.atoms, indices=slice(0, -1))

            train_data_selection = ips.configuration_selection.ThresholdSelection(
                data=md, n_configurations=20, min_distance=20
            )

            test_data_selection = ips.configuration_selection.RandomSelection(
                data=md,
                n_configurations=5,
                exclude_configurations=train_data_selection.selected_configurations,
            )

            train_data += ref_geopt.atoms
            train_data += ips.calculators.CP2KSinglePoint(
                data=train_data_selection.atoms,
                cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                cp2k_shell=cp2k_shell,
                wfn_restart_node=None if cycle > 1 else cp2k,
            ).atoms

            test_data += ips.calculators.CP2KSinglePoint(
                data=test_data_selection.atoms,
                cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
                cp2k_shell=cp2k_shell,
                wfn_restart_node=None if cycle > 1 else cp2k,
            ).atoms

# Isolated Structure Generation

with project:
    with project.group("isolated_mol") as isolated_mol:
        conformers = ips.configuration_generation.SmilesToConformers(smiles="CS(=O)C", numConfs=200)
        isolated_mol_cp2k = ips.calculators.CP2KSinglePoint(
            data=conformers.atoms,
            cp2k_files=["BASIS_MOLOPT", "GTH_POTENTIALS", "dftd3.dat"],
            cp2k_shell=cp2k_shell,
        )

        test_selection = ips.configuration_selection.RandomSelection(
            data=isolated_mol_cp2k.atoms, n_configurations=40,
        )

        test_data_2 =  test_data + test_selection.atoms
        train_data += test_selection.excluded_atoms

        model1 = ips.models.Apax(
                data=train_data,
                validation_data=test_data_2,
                config="config/apax_al2_1.yaml",
        )
        model2 = ips.models.Apax(
            data=train_data,
            validation_data=test_data_2,
            config="config/apax_al2_2.yaml",
        )

        ensemble_model = ips.models.ApaxEnsemble(models=[model1, model2])

        models.append(ensemble_model)


# AL evaluation

In [12]:
with project:
    with project.group("evaluation") as evaluation:
        for model in models:
            prediction = ips.analysis.Prediction(model=model, data=test_data)
            metrics = ips.analysis.PredictionMetrics(data=prediction)

            force_decomposition = ips.analysis.ForceDecomposition(data=prediction)
            volume_scan = ips.analysis.BoxScale(
                data=vs_md.atoms,
                mapping=mapping,
                model=model,
                start=0.9,
                stop=1.7,
                num=50,
                data_id=-1,
            )

# Final Model

In [13]:
with project.group("final") as final:
    model = ips.models.Apax(
            data=train_data,
            validation_data=test_data,
            config="config/apax_final.yaml",
        )
    # evaluate the model
    prediction = ips.analysis.Prediction(model=model, data=test_data)
    metrics = ips.analysis.PredictionMetrics(data=prediction)

    force_decomposition = ips.analysis.ForceDecomposition(data=prediction)
    volume_scan = ips.analysis.BoxScale(
        data=geopt.atoms, mapping=mapping, model=model, start=0.9, data_id=-1
    )

# Build the graph

In [14]:
project.build(nodes=[final])

Running DVC command: 'stage add --name final_MLModel --force ...'
 

 

Running DVC command: 'stage add --name final_Prediction --force ...'


 

 Running DVC command: 'stage add --name final_PredictionMetrics --force ...'
 Running DVC command: 'stage add --name final_ForceDecomposition --force ...'


 

 Running DVC command: 'stage add --name final_BoxScale --force ...'


 

 

 